# California Wildfire Perimeter Analysis

## A Comprehensive Exploratory Data Analysis for Wildfire Risk Modeling

**Prepared for**: Tam Air Club (Tamalpais High School)  
**Collaboration Partners**: UCSF, UCI, CAL FIRE  
**Dataset**: CAL FIRE FRAP Historical Fire Perimeters (1878-2025)  
**Analysis Period**: Focus on 1993-Present (High-Quality GPS Era)

---

### What This Notebook Contains

This comprehensive analysis examines **147 years of California wildfire history** using official CAL FIRE perimeter data. The notebook is organized into 9 parts:

| Part | Topic | Key Outputs |
|------|-------|-------------|
| **2** | Data Quality Assessment | Why 1993+ data is used for modeling |
| **3** | Temporal Analysis | 125-year trends, acceleration since 2000 |
| **4** | Seasonal Patterns | Fire clock, monthly distributions, fire seasons |
| **5** | Fire Size Analysis | Size distributions, Pareto principle, mega-fires |
| **6** | Spatial Analysis | Maps, cumulative burn frequency, risk hotspots |
| **7** | Fire Causes | Known vs unknown causes, investigation challenges |
| **8** | Agency & Unit Analysis | CAL FIRE units, federal vs state jurisdiction |
| **9** | ML Readiness | Data quality assessment for prediction modeling |

### Project Context

This analysis supports **Phase 1** of a collaborative wildfire prediction project. The goal is to build a machine learning model that predicts wildfire risk at **800m × 800m resolution** across California, varying by date and recent conditions.

## Learning Objectives

By completing this analysis, you will be able to:

### Data Quality & Collection
- Explain why CAL FIRE uses **1993+ data** for fire hazard severity zone (FHSZ) mapping
- Understand how GPS adoption transformed fire perimeter data quality
- Interpret collection method codes (GPS Ground, GPS Air, Hand Drawn, etc.)

### Temporal Patterns
- Describe the **dramatic acceleration** of wildfire activity since 2000
- Analyze 125+ years of fire trends using rolling averages and regression
- Compare fire activity across decades (1950s vs 2020s)

### Seasonal Patterns
- Define California's **three fire seasons**: High Risk (Jun-Sep), Transition (Oct-Jan), Low Risk (Feb-May)
- Read and interpret a "Fire Clock" polar visualization
- Understand seasonal heatmaps showing year × month patterns

### Fire Size & Distribution
- Apply the **Pareto Principle** (80/20 rule) to wildfire analysis
- Explain why mega-fires (>100K acres) dominate total burned area
- Interpret log-scale distributions for heavy-tailed data

### Spatial Analysis
- Read a **cumulative burn frequency map** showing fire risk hotspots
- Identify geographic patterns (Northern CA forests, Southern CA chaparral)
- Understand how historical burn data informs future risk prediction

### Causes & Response
- Compare **lightning vs human-caused** fires (count vs burned area)
- Identify which agencies respond to the most fires (CAL FIRE, USDA Forest Service, etc.)
- Interpret fire activity by CAL FIRE administrative unit

### ML Readiness
- Assess data quality requirements for machine learning models
- Understand feature engineering opportunities from fire perimeter data
- Recognize the class imbalance challenge in fire prediction

## Key Questions Answered in This Notebook

### Part 2: Data Quality
- How complete is the historical fire record?
- Why is 1993 the data quality threshold?
- How did collection methods evolve over time?

### Part 3: Temporal Analysis
- **Are California wildfires getting worse?** → Yes, dramatically since 2000
- **When did fires start accelerating?** → Clear inflection point around 2000
- **How have fires changed decade by decade?** → Both count and size increasing

### Part 4: Seasonal Patterns
- **When is fire season in California?** → Peak June-September (~84% of burned area)
- **What defines high-risk vs low-risk periods?** → 4-month seasonal groupings
- **How consistent are seasonal patterns year-to-year?** → Very consistent

### Part 5: Fire Size
- **Do all fires matter equally?** → No, fire damage is extremely concentrated
- **What's the concentration of damage?** → Top 1% of fires cause ~58% of burned area; Top 10% cause ~93%
- **How are fire sizes distributed?** → Heavy-tailed (log-normal)

### Part 6: Spatial Analysis
- **Where do fires burn most frequently?** → Northern CA, Sierra foothills
- **Which areas have burned multiple times?** → Cumulative risk map shows hotspots
- **How can we visualize 30+ years of fire history?** → Burn frequency overlay

### Part 7: Causes
- **What causes California wildfires?** → ~30% have unknown/unidentified causes
- **Why are so many causes unknown?** → Fire investigation is extremely difficult
- **Among known causes, what's most common?** → Lightning (~20%), followed by equipment use and miscellaneous

### Part 8: Agency & Units
- **Who responds to California wildfires?** → CAL FIRE, USDA Forest Service, local agencies
- **Which CAL FIRE units have the most activity?** → Regional breakdown provided
- **How has agency activity changed over time?** → Trends by agency and unit

### Part 9: ML Readiness
- **Is this data ready for machine learning?** → Yes, with >97% completeness for key fields
- **What features can we engineer?** → Temporal, spatial, historical burn frequency
- **What challenges exist?** → Class imbalance, non-stationarity

---

# Part 2: Data Loading & Quality Assessment

Understanding data quality is essential before any analysis. We'll explore why CAL FIRE uses 1993+ data for their official fire hazard severity zone mapping.

In [ ]:
# Standard library imports
import sys
import warnings
from pathlib import Path

# Data manipulation
import numpy as np
import pandas as pd

# Geospatial
import geopandas as gpd
from shapely.geometry import Point
from rasterio.features import rasterize
from rasterio.transform import from_bounds

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap, Normalize
from matplotlib.animation import FuncAnimation, PillowWriter
import seaborn as sns

# Interactive maps
import folium
from folium.plugins import HeatMap

# Basemaps
import contextily as ctx

# Statistical analysis
from scipy import stats
from scipy.stats import pearsonr, spearmanr

# IPython display
from IPython.display import display, HTML

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({
    'figure.figsize': (12, 8),
    'font.size': 11,
    'axes.labelsize': 12,
    'axes.titlesize': 14,
    'figure.dpi': 100
})

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / 'src'))

# Import our custom utilities
from utils.constants import *
from utils.data_loader import *
from utils.plotting import *
from utils.domain_mappings import *

print("All libraries imported successfully!")
print(f"Project root: {project_root}")

## 2.1 Load Fire Perimeter Dataset

In [ ]:
# Load the complete fire perimeters dataset
print("Loading fire perimeters dataset...")
gdf_raw = load_fire_perimeters()
print(f"Loaded {len(gdf_raw):,} fire perimeter records")

# Add derived columns (month, duration, decade, size category)
print("Adding derived columns...")
gdf = add_derived_columns(gdf_raw)

# Add human-readable labels
print("Adding domain labels...")
gdf = apply_domain_labels(gdf)

# Load California boundary for maps
CA_BOUNDARY_PATH = Path("/Users/olivier/Documents/MyQGIS/WildfireAnalysis/DeepLearning/INPUTS_EPSG3310/California_State_Boundary_Reprojected.gpkg")
if CA_BOUNDARY_PATH.exists():
    ca_boundary = gpd.read_file(CA_BOUNDARY_PATH)
    ca_boundary = ca_boundary.to_crs(CRS_ALBERS)
    print(f"California boundary loaded")
else:
    ca_boundary = None
    print("Warning: CA boundary not available")

# Create output directories
FIGURES_DIR.mkdir(parents=True, exist_ok=True)
(FIGURES_DIR / 'comprehensive').mkdir(exist_ok=True)
INTERACTIVE_DIR = OUTPUT_DIR / 'interactive'
INTERACTIVE_DIR.mkdir(exist_ok=True)

print(f"\nDataset ready: {len(gdf):,} records spanning {gdf['YEAR_'].min()}-{gdf['YEAR_'].max()}")

## 2.2 Schema Exploration

In [ ]:
# Display dataset schema
print("=== Fire Perimeter Dataset Schema ===\n")
print(f"Total Records: {len(gdf):,}")
print(f"Year Range: {gdf['YEAR_'].min()} - {gdf['YEAR_'].max()}")
print(f"CRS: {gdf.crs}")
print(f"\nColumns ({len(gdf.columns)}):")
print("-" * 50)

for col in gdf.columns[:20]:  # Show first 20 columns
    dtype = gdf[col].dtype
    non_null = gdf[col].notna().sum()
    pct = non_null / len(gdf) * 100
    print(f"  {col:20} {str(dtype):15} ({pct:5.1f}% complete)")

## 2.3 The 1993 Data Quality Threshold

**Why 1993?** CAL FIRE's data collection dramatically improved in 1993 with the adoption of GPS technology for perimeter mapping. Before 1993, fire perimeters were primarily:

- Hand-drawn from paper maps
- Digitized from aerial photo interpretation
- Often missing key attributes (cause, dates, agency)

Let's compare data completeness before and after 1993:

In [ ]:
# Split data at 1993 threshold
gdf_pre1993 = gdf[gdf['YEAR_'] < 1993]
gdf_hq = gdf[gdf['YEAR_'] >= 1993]  # High-quality data

print(f"Pre-1993 records: {len(gdf_pre1993):,} ({len(gdf_pre1993)/len(gdf)*100:.1f}%)")
print(f"1993+ records:    {len(gdf_hq):,} ({len(gdf_hq)/len(gdf)*100:.1f}%)")

# Key fields to compare
key_fields = ['CAUSE', 'AGENCY', 'FIRE_NAME', 'ALARM_DATE', 'CONT_DATE', 'GIS_ACRES']

# Calculate COMPLETENESS (not missing) for each field
completeness_data = []
for field in key_fields:
    pre_complete = gdf_pre1993[field].notna().mean() * 100
    post_complete = gdf_hq[field].notna().mean() * 100
    completeness_data.append({
        'Field': field,
        'Pre-1993': pre_complete,
        '1993+': post_complete,
        'Improvement': post_complete - pre_complete
    })

completeness_df = pd.DataFrame(completeness_data)
print("\n=== Data Completeness Comparison ===")
print(completeness_df.to_string(index=False))

In [ ]:
# Figure 2.4a: Data Completeness Comparison (Dual Panel)
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Color function based on completeness
def get_quality_color(pct):
    if pct >= 90:
        return '#2ECC71'  # Green - Excellent
    elif pct >= 70:
        return '#F1C40F'  # Yellow - Good
    else:
        return '#E74C3C'  # Red - Poor

fields = completeness_df['Field'].values
y_pos = np.arange(len(fields))

# Left panel: Pre-1993
ax1 = axes[0]
pre_values = completeness_df['Pre-1993'].values
colors_pre = [get_quality_color(v) for v in pre_values]
bars1 = ax1.barh(y_pos, pre_values, color=colors_pre, edgecolor='white', height=0.7)
ax1.set_yticks(y_pos)
ax1.set_yticklabels(fields)
ax1.set_xlabel('Data Completeness (%)', fontsize=12)
ax1.set_title(f'Pre-1993 Data Quality\n({len(gdf_pre1993):,} records)', fontsize=14, fontweight='bold')
ax1.set_xlim(0, 105)
ax1.axvline(x=90, color='green', linestyle='--', alpha=0.5, label='Excellent (90%)')
ax1.axvline(x=70, color='orange', linestyle='--', alpha=0.5, label='Good (70%)')

# Add value labels
for i, (bar, val) in enumerate(zip(bars1, pre_values)):
    ax1.text(val + 2, i, f'{val:.0f}%', va='center', fontsize=10)

# Right panel: 1993+
ax2 = axes[1]
post_values = completeness_df['1993+'].values
colors_post = [get_quality_color(v) for v in post_values]
bars2 = ax2.barh(y_pos, post_values, color=colors_post, edgecolor='white', height=0.7)
ax2.set_yticks(y_pos)
ax2.set_yticklabels(fields)
ax2.set_xlabel('Data Completeness (%)', fontsize=12)
ax2.set_title(f'1993+ Data Quality (GPS Era)\n({len(gdf_hq):,} records)', fontsize=14, fontweight='bold')
ax2.set_xlim(0, 105)
ax2.axvline(x=90, color='green', linestyle='--', alpha=0.5)
ax2.axvline(x=70, color='orange', linestyle='--', alpha=0.5)

# Add value labels
for i, (bar, val) in enumerate(zip(bars2, post_values)):
    ax2.text(val + 2, i, f'{val:.0f}%', va='center', fontsize=10)

# Legend
legend_elements = [
    mpatches.Patch(facecolor='#2ECC71', label='Excellent (≥90%)'),
    mpatches.Patch(facecolor='#F1C40F', label='Good (70-90%)'),
    mpatches.Patch(facecolor='#E74C3C', label='Poor (<70%)')
]
fig.legend(handles=legend_elements, loc='lower center', ncol=3, fontsize=11, 
           bbox_to_anchor=(0.5, -0.02))

plt.suptitle('Why CAL FIRE Uses 1993+ Data for Risk Modeling\nGPS Adoption Dramatically Improved Data Quality', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()

# Save
save_path = FIGURES_DIR / 'comprehensive' / '01_data_completeness_comparison.png'
fig.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print(f"Figure saved: {save_path}")

In [ ]:
# Figure 2.4b: Collection Method Evolution Over Time
fig, ax = plt.subplots(figsize=(14, 6))

# Get collection method by decade
decade_method = gdf.groupby(['DECADE', 'C_METHOD']).size().unstack(fill_value=0)

# Map codes to labels
method_labels = {1: 'GPS Ground', 2: 'GPS Air', 3: 'Infrared', 4: 'Other Imagery',
                 5: 'Photo Interp', 6: 'Hand Drawn', 7: 'Mixed', 8: 'Unknown'}

# Reorder columns for logical display (modern to historical)
ordered_methods = [1, 2, 3, 4, 5, 6, 7, 8]
ordered_methods = [m for m in ordered_methods if m in decade_method.columns]

# Colors: GPS methods in green, traditional in red/orange
method_colors = {
    1: '#27AE60', 2: '#2ECC71',  # GPS - Green
    3: '#3498DB', 4: '#5DADE2',  # Modern imaging - Blue
    5: '#F39C12', 6: '#E74C3C',  # Traditional - Orange/Red
    7: '#95A5A6', 8: '#BDC3C7'   # Mixed/Unknown - Gray
}

colors = [method_colors.get(m, '#95A5A6') for m in ordered_methods]
labels = [method_labels.get(m, f'Code {m}') for m in ordered_methods]

decade_method[ordered_methods].plot(kind='bar', stacked=True, ax=ax, 
                                     color=colors, width=0.8, edgecolor='white')

ax.set_xlabel('Decade', fontsize=12)
ax.set_ylabel('Number of Fire Records', fontsize=12)
ax.set_title('Evolution of Fire Perimeter Collection Methods\nTransition from Hand-Drawn to GPS-Based Mapping', 
             fontsize=14, fontweight='bold')
ax.legend(labels, title='Collection Method', bbox_to_anchor=(1.02, 1), loc='upper left')

# Find the position of the 1990s decade for the 1993 annotation
decades_list = list(decade_method.index)
if 1990 in decades_list:
    pos_1990 = decades_list.index(1990)
    # Place line at start of 1990s bar (1993 is early in the decade)
    ax.axvline(x=pos_1990 - 0.2, color='red', linestyle='--', linewidth=2, alpha=0.7)
    ax.text(pos_1990 - 0.1, ax.get_ylim()[1]*0.9, '1993\nGPS\nAdoption', 
            fontsize=10, color='red', fontweight='bold')

plt.xticks(rotation=45)
plt.tight_layout()

save_path = FIGURES_DIR / 'comprehensive' / '02_collection_method_evolution.png'
fig.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print(f"Figure saved: {save_path}")

### Key Takeaway: Why 1993+ Data

**CAL FIRE uses 1993+ data for fire hazard severity zone (FHSZ) mapping because:**

1. **GPS Accuracy**: Fire perimeter boundaries are precise (not estimated from paper maps)
2. **Complete Attribution**: >90% of records have cause, agency, dates, and size information
3. **Consistent Standards**: Standardized data collection protocols were established
4. **Sufficient History**: 30+ years provides robust patterns for statistical analysis

**For our ML model, we will use 1993+ data as the training dataset.**

---
# Part 3: Temporal Analysis

**Question: "Are California wildfires getting worse?"**

This section consolidates all temporal analysis into a single comprehensive view. We'll examine:
1. Long-term historical trends (147 years of data)
2. The acceleration of fires since 2000
3. Decade-by-decade comparisons
4. Statistical trend analysis

## 3.1 "Are California wildfires getting worse?"

Let's visualize 147 years of California fire history. We'll show both fire count (how many fires) and burned area (how much land burned).

In [ ]:
# Figure 3.1: Full Historical Timeline (Dual Panel)
# Data: ALL YEARS 1900-Present (includes variable quality pre-1993 data)
gdf_modern = gdf[gdf['YEAR_'] >= 1900]
max_year = gdf_modern['YEAR_'].max()

# Aggregate by year
yearly_stats = gdf_modern.groupby('YEAR_').agg({
    'OBJECTID': 'count',  # Fire count
    'GIS_ACRES': 'sum'    # Total acres burned
}).rename(columns={'OBJECTID': 'fire_count', 'GIS_ACRES': 'acres_burned'})
yearly_stats = yearly_stats.reset_index()

# Calculate 5-year rolling average
yearly_stats['count_rolling'] = yearly_stats['fire_count'].rolling(window=5, center=True).mean()
yearly_stats['acres_rolling'] = yearly_stats['acres_burned'].rolling(window=5, center=True).mean()

# Find worst year
worst_year_idx = yearly_stats['acres_burned'].idxmax()
worst_year = yearly_stats.loc[worst_year_idx, 'YEAR_']
worst_acres = yearly_stats.loc[worst_year_idx, 'acres_burned']

# Create dual panel figure
fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# Left Panel: Fire Count
ax1 = axes[0]
ax1.bar(yearly_stats['YEAR_'], yearly_stats['fire_count'], color='#3498DB', alpha=0.7, label='Annual Count')
ax1.plot(yearly_stats['YEAR_'], yearly_stats['count_rolling'], color='#E74C3C', linewidth=2.5, label='5-Year Average')

# Key annotations
ax1.axvline(x=1993, color='green', linestyle='--', linewidth=2, alpha=0.7)
ax1.axvline(x=2000, color='orange', linestyle='--', linewidth=2, alpha=0.7)

ax1.set_xlabel('Year', fontsize=12)
ax1.set_ylabel('Number of Fires', fontsize=12)
ax1.set_title(f'Fire Count by Year\nData: 1900-{max_year} (All Available Records)', fontsize=14, fontweight='bold')
ax1.legend(loc='upper left')
ax1.set_xlim(1900, max_year + 2)

# Add annotation box
ax1.text(0.02, 0.98, 'Green line: 1993 data quality threshold\nOrange line: 2000 acceleration point', 
         transform=ax1.transAxes, fontsize=9, va='top', 
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Right Panel: Burned Area - SHOW ALL DATA (no cropping)
ax2 = axes[1]

# Plot all bars without cropping
ax2.bar(yearly_stats['YEAR_'], yearly_stats['acres_burned'] / 1e6, color='#E74C3C', alpha=0.7, label='Annual Burned')
ax2.plot(yearly_stats['YEAR_'], yearly_stats['acres_rolling'] / 1e6, color='#2C3E50', linewidth=2.5, label='5-Year Average')

# Let matplotlib auto-scale to show ALL data
ax2.set_ylim(0, yearly_stats['acres_burned'].max() / 1e6 * 1.1)

# Annotate worst year
ax2.annotate(f'{int(worst_year)}\n{worst_acres/1e6:.1f}M acres', 
             xy=(worst_year, worst_acres/1e6), 
             xytext=(worst_year-12, worst_acres/1e6 * 0.85),
             fontsize=10, fontweight='bold', color='darkred',
             arrowprops=dict(arrowstyle='->', color='darkred'))

ax2.axvline(x=1993, color='green', linestyle='--', linewidth=2, alpha=0.7)
ax2.axvline(x=2000, color='orange', linestyle='--', linewidth=2, alpha=0.7)

ax2.set_xlabel('Year', fontsize=12)
ax2.set_ylabel('Million Acres Burned', fontsize=12)
ax2.set_title(f'Total Burned Area by Year\nData: 1900-{max_year} (All Data Shown)', fontsize=14, fontweight='bold')
ax2.legend(loc='upper left')
ax2.set_xlim(1900, max_year + 2)

plt.suptitle(f'California Wildfires 1900-{max_year}: Fire Activity Has Dramatically Increased', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()

save_path = FIGURES_DIR / 'comprehensive' / '03_temporal_timeline_dual.png'
fig.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print(f"Figure saved: {save_path}")

# Print statistics
print(f"\n=== Key Statistics (1900-{max_year}) ===")
print(f"Total fires: {yearly_stats['fire_count'].sum():,}")
print(f"Total acres burned: {yearly_stats['acres_burned'].sum()/1e6:.2f} million acres")
print(f"Worst fire year: {int(worst_year)} ({worst_acres/1e6:.2f}M acres)")
print(f"Note: Pre-1993 data has variable quality")

## 3.2 "When did fires start accelerating?"

Scientists and fire managers have noted a dramatic shift in fire behavior around the year 2000. Let's use statistical regression to quantify this acceleration.

In [ ]:
# Figure 3.2: Trend Analysis - Pre/Post-2000 Regression (1950-Present)
# Use extended historical data from 1950 to capture the full acceleration pattern
gdf_1950 = gdf[gdf['YEAR_'] >= 1950]

yearly_extended = gdf_1950.groupby('YEAR_').agg({
    'OBJECTID': 'count',
    'GIS_ACRES': 'sum'
}).rename(columns={'OBJECTID': 'fire_count', 'GIS_ACRES': 'acres_burned'}).reset_index()

# Split at 2000 to compare eras
pre_2000 = yearly_extended[yearly_extended['YEAR_'] < 2000]
post_2000 = yearly_extended[yearly_extended['YEAR_'] >= 2000]

fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Left Panel: Fire Count Trend
ax1 = axes[0]

# Scatter points with different colors for eras
ax1.scatter(pre_2000['YEAR_'], pre_2000['fire_count'], color='#3498DB', s=60, alpha=0.7, label='1950-1999', zorder=5)
ax1.scatter(post_2000['YEAR_'], post_2000['fire_count'], color='#E74C3C', s=60, alpha=0.7, label='2000-Present', zorder=5)

# Regression lines
if len(pre_2000) > 1:
    slope_pre, intercept_pre, r_pre, p_pre, se_pre = stats.linregress(pre_2000['YEAR_'], pre_2000['fire_count'])
    x_pre = np.array([pre_2000['YEAR_'].min(), pre_2000['YEAR_'].max()])
    ax1.plot(x_pre, slope_pre * x_pre + intercept_pre, 
             color='#2980B9', linewidth=3, linestyle='--', label=f'1950-1999 trend: {slope_pre:+.1f}/year')

if len(post_2000) > 1:
    slope_post, intercept_post, r_post, p_post, se_post = stats.linregress(post_2000['YEAR_'], post_2000['fire_count'])
    x_post = np.array([post_2000['YEAR_'].min(), post_2000['YEAR_'].max()])
    ax1.plot(x_post, slope_post * x_post + intercept_post, 
             color='#C0392B', linewidth=3, linestyle='--', label=f'2000+ trend: {slope_post:+.1f}/year')

ax1.axvline(x=2000, color='black', linestyle=':', linewidth=2, alpha=0.7)
ax1.text(2000, ax1.get_ylim()[1]*0.95, '2000\nInflection', ha='center', fontsize=9, fontweight='bold')
ax1.set_xlabel('Year', fontsize=12)
ax1.set_ylabel('Number of Fires', fontsize=12)
ax1.set_title('Fire Count Trend Analysis (1950-Present)\n50 years before vs 25 years after 2000', fontsize=14, fontweight='bold')
ax1.legend(loc='upper left', fontsize=9)
ax1.set_xlim(1950, 2030)

# Right Panel: Burned Area Trend
ax2 = axes[1]

ax2.scatter(pre_2000['YEAR_'], pre_2000['acres_burned']/1e6, color='#3498DB', s=60, alpha=0.7, label='1950-1999', zorder=5)
ax2.scatter(post_2000['YEAR_'], post_2000['acres_burned']/1e6, color='#E74C3C', s=60, alpha=0.7, label='2000-Present', zorder=5)

# Regression for acres
if len(pre_2000) > 1:
    slope_pre_a, intercept_pre_a, r_pre_a, p_pre_a, se_pre_a = stats.linregress(pre_2000['YEAR_'], pre_2000['acres_burned']/1e6)
    ax2.plot(x_pre, slope_pre_a * x_pre + intercept_pre_a, 
             color='#2980B9', linewidth=3, linestyle='--', label=f'1950-1999: {slope_pre_a*1000:+.0f}K acres/year')

if len(post_2000) > 1:
    slope_post_a, intercept_post_a, r_post_a, p_post_a, se_post_a = stats.linregress(post_2000['YEAR_'], post_2000['acres_burned']/1e6)
    ax2.plot(x_post, slope_post_a * x_post + intercept_post_a, 
             color='#C0392B', linewidth=3, linestyle='--', label=f'2000+: {slope_post_a*1000:+.0f}K acres/year')

ax2.axvline(x=2000, color='black', linestyle=':', linewidth=2, alpha=0.7)
ax2.text(2000, ax2.get_ylim()[1]*0.95, '2000\nInflection', ha='center', fontsize=9, fontweight='bold')
ax2.set_xlabel('Year', fontsize=12)
ax2.set_ylabel('Million Acres Burned', fontsize=12)
ax2.set_title('Burned Area Trend Analysis (1950-Present)\nDramatic acceleration after 2000', fontsize=14, fontweight='bold')
ax2.legend(loc='upper left', fontsize=9)
ax2.set_xlim(1950, 2030)

plt.suptitle('75 Years of Data Reveal Clear Acceleration: Fire Activity Changed Dramatically After 2000', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()

save_path = FIGURES_DIR / 'comprehensive' / '04_temporal_trend_analysis.png'
fig.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print(f"Figure saved: {save_path}")

# Calculate acceleration factor
print("\n=== Trend Analysis Results (1950-Present) ===")
print(f"Fire Count Trend:")
print(f"  1950-1999 (50 years): {slope_pre:+.2f} fires/year")
print(f"  2000-Present (25 years): {slope_post:+.2f} fires/year")
if slope_pre != 0:
    print(f"  Acceleration factor: {abs(slope_post/slope_pre):.1f}x {'faster increase' if slope_post > slope_pre else 'change'}")
print(f"\nBurned Area Trend:")
print(f"  1950-1999: {slope_pre_a*1000:+.0f} thousand acres/year")
print(f"  2000-Present: {slope_post_a*1000:+.0f} thousand acres/year")
if slope_pre_a != 0:
    print(f"  Acceleration factor: {abs(slope_post_a/slope_pre_a):.1f}x {'faster increase' if slope_post_a > slope_pre_a else 'change'}")

# Additional statistics
print(f"\n=== Era Comparison ===")
print(f"Pre-2000 average fires/year: {pre_2000['fire_count'].mean():.0f}")
print(f"Post-2000 average fires/year: {post_2000['fire_count'].mean():.0f}")
print(f"Pre-2000 average acres/year: {pre_2000['acres_burned'].mean()/1e6:.2f}M")
print(f"Post-2000 average acres/year: {post_2000['acres_burned'].mean()/1e6:.2f}M")

## 3.3 "How have fires changed decade by decade?"

Breaking down the data by decade reveals how fire patterns have evolved over time.

In [ ]:
# Figure 3.3: Decade Comparison (2x2 Grid)
# Data: HIGH-QUALITY ONLY (1993-Present) for reliable comparisons
min_year_hq = gdf_hq['YEAR_'].min()
max_year_hq = gdf_hq['YEAR_'].max()

decade_stats = gdf_hq.groupby('DECADE').agg({
    'OBJECTID': 'count',
    'GIS_ACRES': ['sum', 'mean'],
}).reset_index()
decade_stats.columns = ['DECADE', 'fire_count', 'total_acres', 'mean_acres']

# Count mega-fires by decade
mega_fires = gdf_hq[gdf_hq['GIS_ACRES'] >= 100000].groupby('DECADE').size().reset_index(name='mega_fire_count')
decade_stats = decade_stats.merge(mega_fires, on='DECADE', how='left')
decade_stats['mega_fire_count'] = decade_stats['mega_fire_count'].fillna(0)

# Create 2x2 grid
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
decades = decade_stats['DECADE'].values
x_pos = np.arange(len(decades))

# Define colors - progressively more intense
decade_colors = plt.cm.YlOrRd(np.linspace(0.3, 0.9, len(decades)))

# Panel 1: Fire Count
ax1 = axes[0, 0]
bars1 = ax1.bar(x_pos, decade_stats['fire_count'], color=decade_colors, edgecolor='white')
ax1.set_xticks(x_pos)
ax1.set_xticklabels([f"{int(d)}s" for d in decades])
ax1.set_ylabel('Number of Fires')
ax1.set_title('Total Fire Count by Decade', fontsize=12, fontweight='bold')
for i, v in enumerate(decade_stats['fire_count']):
    ax1.text(i, v + 50, f'{int(v):,}', ha='center', fontsize=9)

# Panel 2: Total Acres Burned
ax2 = axes[0, 1]
bars2 = ax2.bar(x_pos, decade_stats['total_acres']/1e6, color=decade_colors, edgecolor='white')
ax2.set_xticks(x_pos)
ax2.set_xticklabels([f"{int(d)}s" for d in decades])
ax2.set_ylabel('Million Acres Burned')
ax2.set_title('Total Burned Area by Decade', fontsize=12, fontweight='bold')
for i, v in enumerate(decade_stats['total_acres']/1e6):
    ax2.text(i, v + 0.2, f'{v:.1f}M', ha='center', fontsize=9)

# Panel 3: Mean Fire Size
ax3 = axes[1, 0]
bars3 = ax3.bar(x_pos, decade_stats['mean_acres'], color=decade_colors, edgecolor='white')
ax3.set_xticks(x_pos)
ax3.set_xticklabels([f"{int(d)}s" for d in decades])
ax3.set_ylabel('Average Acres per Fire')
ax3.set_title('Average Fire Size by Decade', fontsize=12, fontweight='bold')
for i, v in enumerate(decade_stats['mean_acres']):
    ax3.text(i, v + 100, f'{int(v):,}', ha='center', fontsize=9)

# Panel 4: Mega-Fires
ax4 = axes[1, 1]
bars4 = ax4.bar(x_pos, decade_stats['mega_fire_count'], color=decade_colors, edgecolor='white')
ax4.set_xticks(x_pos)
ax4.set_xticklabels([f"{int(d)}s" for d in decades])
ax4.set_ylabel('Number of Mega-Fires')
ax4.set_title('Mega-Fires (>100K acres) by Decade', fontsize=12, fontweight='bold')
for i, v in enumerate(decade_stats['mega_fire_count']):
    ax4.text(i, v + 0.5, f'{int(v)}', ha='center', fontsize=9, fontweight='bold')

plt.suptitle(f'Decade-by-Decade Fire Analysis\nData: {min_year_hq}-{max_year_hq} (High-Quality GPS Era)', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()

save_path = FIGURES_DIR / 'comprehensive' / '05_decade_comparison.png'
fig.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print(f"Figure saved: {save_path}")

# Print decade statistics
print(f"\n=== Decade Statistics ({min_year_hq}-{max_year_hq}) ===")
print(decade_stats.to_string(index=False))

## 3.4 Key Takeaways: Temporal Patterns

**What the data tells us:**

1. **California wildfires ARE getting worse** - Both fire count and burned area show clear upward trends
2. **The year 2000 was an inflection point** - Fire activity accelerated dramatically after 2000
3. **Mega-fires are becoming more common** - The 2020s have seen more mega-fires than any previous decade
4. **2020 was the worst year on record** - Over 4 million acres burned in a single year

**ML Model Implications:**
- Year/decade should be included as temporal features
- Consider non-stationarity in the data (climate change effects)
- Train/test split should be temporal, not random
- More recent data may be more predictive of future patterns

---
# Part 4: Seasonal Patterns

**Question: "When is fire season in California?"**

California has distinct fire seasons driven by weather patterns, vegetation dryness, and wind events. We define three 4-month seasons:

- **High Risk Season (June-September)**: Peak fire activity, dry vegetation, hot temperatures
- **Transition Season (October-January)**: Santa Ana winds, variable conditions
- **Low Risk Season (February-May)**: Wet season, green vegetation

In [ ]:
# Define fire seasons with semantic colors
def get_fire_season(month):
    """Classify month into fire season (4-month groupings)."""
    if month in [6, 7, 8, 9]:       # June-September
        return 'High Risk Season'
    elif month in [10, 11, 12, 1]:  # October-January
        return 'Transition Season'
    else:                            # February-May
        return 'Low Risk Season'

# Semantic color scheme
SEASON_COLORS = {
    'High Risk Season': '#d73027',      # Red
    'Transition Season': '#fee090',     # Yellow/Orange
    'Low Risk Season': '#4575b4'        # Blue
}

# Month to season mapping for plotting
MONTH_SEASONS = {
    1: 'Transition Season', 2: 'Low Risk Season', 3: 'Low Risk Season',
    4: 'Low Risk Season', 5: 'Low Risk Season', 6: 'High Risk Season',
    7: 'High Risk Season', 8: 'High Risk Season', 9: 'High Risk Season',
    10: 'Transition Season', 11: 'Transition Season', 12: 'Transition Season'
}

# Add fire season to data
gdf_hq['FIRE_SEASON'] = gdf_hq['ALARM_MONTH'].apply(get_fire_season)

# Calculate monthly statistics for high-quality data
monthly_stats = gdf_hq.groupby('ALARM_MONTH').agg({
    'OBJECTID': 'count',
    'GIS_ACRES': 'sum'
}).rename(columns={'OBJECTID': 'fire_count', 'GIS_ACRES': 'acres_burned'}).reset_index()

# Add season info
monthly_stats['season'] = monthly_stats['ALARM_MONTH'].map(MONTH_SEASONS)
monthly_stats['color'] = monthly_stats['season'].map(SEASON_COLORS)

print("=== Fire Season Definition ===")
print("High Risk Season: June, July, August, September")
print("Transition Season: October, November, December, January")
print("Low Risk Season: February, March, April, May")
print(f"\nFires by season (1993+):")
season_counts = gdf_hq['FIRE_SEASON'].value_counts()
for season, count in season_counts.items():
    pct = count / len(gdf_hq) * 100
    print(f"  {season}: {count:,} fires ({pct:.1f}%)")

## 4.1 Monthly Fire Distribution

The seasonal pattern is clearly visible in monthly fire counts and burned area. Colors indicate fire season risk level.

In [ ]:
# Figure 4.1: Monthly Distribution (Dual Panel with Semantic Colors)
# Data: HIGH-QUALITY ONLY (1993-Present)
month_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Left Panel: Fire Count by Month
ax1 = axes[0]
bars1 = ax1.bar(monthly_stats['ALARM_MONTH'], monthly_stats['fire_count'], 
                color=monthly_stats['color'], edgecolor='white', width=0.8)
ax1.set_xticks(range(1, 13))
ax1.set_xticklabels(month_labels)
ax1.set_xlabel('Month', fontsize=12)
ax1.set_ylabel('Number of Fires', fontsize=12)
ax1.set_title(f'Fire Count by Month\nData: {min_year_hq}-{max_year_hq} (High-Quality)', fontsize=14, fontweight='bold')

# Add value labels
for i, v in enumerate(monthly_stats['fire_count']):
    ax1.text(i+1, v + 50, f'{int(v):,}', ha='center', fontsize=9)

# Right Panel: Burned Area by Month
ax2 = axes[1]
bars2 = ax2.bar(monthly_stats['ALARM_MONTH'], monthly_stats['acres_burned']/1e6, 
                color=monthly_stats['color'], edgecolor='white', width=0.8)
ax2.set_xticks(range(1, 13))
ax2.set_xticklabels(month_labels)
ax2.set_xlabel('Month', fontsize=12)
ax2.set_ylabel('Million Acres Burned', fontsize=12)
ax2.set_title(f'Total Burned Area by Month\nData: {min_year_hq}-{max_year_hq} (High-Quality)', fontsize=14, fontweight='bold')

# Add value labels
for i, v in enumerate(monthly_stats['acres_burned']/1e6):
    ax2.text(i+1, v + 0.2, f'{v:.1f}M', ha='center', fontsize=9)

# Create legend
legend_elements = [
    mpatches.Patch(facecolor='#d73027', label='High Risk (Jun-Sep)'),
    mpatches.Patch(facecolor='#fee090', label='Transition (Oct-Jan)'),
    mpatches.Patch(facecolor='#4575b4', label='Low Risk (Feb-May)')
]
fig.legend(handles=legend_elements, loc='lower center', ncol=3, fontsize=11,
           bbox_to_anchor=(0.5, -0.02))

plt.suptitle(f'California Fire Seasons ({min_year_hq}-{max_year_hq}): A Clear Annual Pattern', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()

save_path = FIGURES_DIR / 'comprehensive' / '06_seasonal_monthly_distribution.png'
fig.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print(f"Figure saved: {save_path}")

## 4.2 The Fire Clock

A polar (circular) plot shows the annual fire cycle more intuitively. Think of it as a clock where each month is a wedge.

In [ ]:
# Figure 4.2: The Fire Clock - A Visual Calendar of California's Fire Seasons
# Data: HIGH-QUALITY ONLY (1993-Present)

fig = plt.figure(figsize=(14, 14))

# Create a single large polar plot
ax = fig.add_subplot(111, projection='polar')

# Ensure monthly_stats is sorted by month
monthly_sorted = monthly_stats.sort_values('ALARM_MONTH').reset_index(drop=True)

# Create angles for 12 months (like a clock, Jan at top, going clockwise)
theta = np.linspace(0, 2*np.pi, 12, endpoint=False)

# Normalize data for visualization
fire_counts = monthly_sorted['fire_count'].values
acres_burned = monthly_sorted['acres_burned'].values

# Normalize to 0-1 scale for overlay
counts_norm = fire_counts / fire_counts.max()
acres_norm = acres_burned / acres_burned.max()

# Bar width
width = 2*np.pi/12 * 0.9

# Season colors for each month
colors = [SEASON_COLORS[MONTH_SEASONS[m]] for m in range(1, 13)]

# Create gradient bars showing fire intensity
# Outer ring: Burned acres (the "danger" metric)
bars_outer = ax.bar(theta, acres_norm * 0.9 + 0.1, width=width, 
                    bottom=0.5,
                    color=colors, edgecolor='white', linewidth=2, alpha=0.85)

# Inner ring: Fire counts
bars_inner = ax.bar(theta, counts_norm * 0.4, width=width * 0.6,
                    bottom=0.1,
                    color=colors, edgecolor='white', linewidth=1, alpha=0.7)

# Configure the polar plot to look like a clock
ax.set_theta_zero_location('N')  # 12 o'clock = January
ax.set_theta_direction(-1)  # Clockwise

# Month labels on the outside
ax.set_xticks(theta)
ax.set_xticklabels(['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 
                    'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC'], 
                   fontsize=12, fontweight='bold')

# Remove radial labels (we'll add custom annotations)
ax.set_yticklabels([])
ax.set_ylim(0, 1.5)

# Add concentric guide circles
for r in [0.5, 1.0, 1.4]:
    circle = plt.Circle((0, 0), r, transform=ax.transData._b, 
                        fill=False, color='gray', linewidth=0.5, alpha=0.3)

# Add season labels in the center area
ax.annotate('FIRE\nCLOCK', xy=(0, 0), ha='center', va='center',
            fontsize=20, fontweight='bold', color='#333')

# Add annotations for peak months
peak_month_idx = np.argmax(acres_burned)
peak_theta = theta[peak_month_idx]
ax.annotate(f'PEAK\n{acres_burned[peak_month_idx]/1e6:.1f}M acres',
            xy=(peak_theta, 1.35), ha='center', va='center',
            fontsize=10, fontweight='bold', color='#d73027',
            bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor='#d73027', alpha=0.9))

# Add data annotations for key months
annotations = [
    (6, 'JUL: Peak fire\ncount begins'),  # July (index 6)
    (8, 'SEP: Worst\nacres burned'),      # September
    (9, 'OCT: Santa Ana\nwind season'),   # October
]

# Legend with actual statistics
legend_text = f"""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━
OUTER RING: Acres Burned
INNER RING: Fire Count  
━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🔴 HIGH RISK: Jun-Sep
   {fire_counts[5:9].sum():,} fires
   {acres_burned[5:9].sum()/1e6:.1f}M acres
   
🟡 TRANSITION: Oct-Jan  
   {fire_counts[9:12].sum() + fire_counts[0]:,} fires
   {(acres_burned[9:12].sum() + acres_burned[0])/1e6:.1f}M acres
   
🔵 LOW RISK: Feb-May
   {fire_counts[1:5].sum():,} fires
   {acres_burned[1:5].sum()/1e6:.1f}M acres
━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Data: {min_year_hq}-{max_year_hq}
"""

# Add legend box
fig.text(0.02, 0.02, legend_text, transform=fig.transFigure,
         fontsize=10, fontfamily='monospace', verticalalignment='bottom',
         bbox=dict(boxstyle='round', facecolor='white', edgecolor='gray', alpha=0.95))

# Season arc annotations
from matplotlib.patches import Arc, FancyArrowPatch
import matplotlib.patches as mpatches

# Create season legend patches
legend_elements = [
    mpatches.Patch(facecolor='#d73027', edgecolor='white', label='High Risk Season (Jun-Sep)'),
    mpatches.Patch(facecolor='#fee090', edgecolor='white', label='Transition Season (Oct-Jan)'),
    mpatches.Patch(facecolor='#4575b4', edgecolor='white', label='Low Risk Season (Feb-May)')
]
fig.legend(handles=legend_elements, loc='upper right', fontsize=11,
           bbox_to_anchor=(0.98, 0.98), framealpha=0.95)

plt.title(f"California's Fire Clock: When Wildfires Strike\n"
          f"30+ Years of Data ({min_year_hq}-{max_year_hq}) | Outer=Acres Burned, Inner=Fire Count",
          fontsize=16, fontweight='bold', pad=20)

plt.tight_layout()

save_path = FIGURES_DIR / 'comprehensive' / '07_fire_clock_polar.png'
fig.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print(f"Figure saved: {save_path}")
print(f"\n📊 The Fire Clock shows California's annual wildfire rhythm:")
print(f"   • High Risk Season (Jun-Sep): {acres_burned[5:9].sum()/acres_burned.sum()*100:.0f}% of all acres burned")
print(f"   • Peak month: {month_labels[np.argmax(acres_burned)]} with {acres_burned.max()/1e6:.1f}M acres")

## 4.3 Year × Month Heatmap

A heatmap reveals year-over-year patterns and helps identify exceptional fire months across the historical record.

In [ ]:
# Figure 4.3: Year × Month Heatmaps (Dual Panel)
# Data: HIGH-QUALITY ONLY (1993-Present)

# Create pivot tables for heatmaps
year_month_count = gdf_hq.groupby(['YEAR_', 'ALARM_MONTH']).size().unstack(fill_value=0)
year_month_acres = gdf_hq.groupby(['YEAR_', 'ALARM_MONTH'])['GIS_ACRES'].sum().unstack(fill_value=0)

fig, axes = plt.subplots(1, 2, figsize=(18, 10))

# Left: Fire Count Heatmap
ax1 = axes[0]
sns.heatmap(year_month_count, cmap='YlOrRd', ax=ax1, cbar_kws={'label': 'Fire Count'})
ax1.set_xticklabels(month_labels)
ax1.set_xlabel('Month', fontsize=12)
ax1.set_ylabel('Year', fontsize=12)
ax1.set_title(f'Fire Count by Year and Month\nData: {min_year_hq}-{max_year_hq}', fontsize=14, fontweight='bold')

# Right: Burned Area Heatmap (log scale for better visibility)
ax2 = axes[1]
acres_log = np.log10(year_month_acres + 1)
sns.heatmap(acres_log, cmap='YlOrRd', ax=ax2, cbar_kws={'label': 'Log₁₀(Acres Burned + 1)'})
ax2.set_xticklabels(month_labels)
ax2.set_xlabel('Month', fontsize=12)
ax2.set_ylabel('Year', fontsize=12)
ax2.set_title(f'Burned Area by Year and Month (log scale)\nData: {min_year_hq}-{max_year_hq}', fontsize=14, fontweight='bold')

plt.suptitle(f'Year × Month Fire Activity Patterns\nData: {min_year_hq}-{max_year_hq} (High-Quality GPS Era)', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()

save_path = FIGURES_DIR / 'comprehensive' / '08_seasonal_heatmap.png'
fig.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print(f"Figure saved: {save_path}")

## 4.4 Fire Season Statistics

Let's quantify the difference between our three fire seasons in terms of fire count, burned area, and average fire size.

In [ ]:
# Figure 4.4: Fire Season Comparison (3 Metrics)
# Data: HIGH-QUALITY ONLY (1993-Present)

# Calculate statistics by season
season_stats = gdf_hq.groupby('FIRE_SEASON').agg({
    'OBJECTID': 'count',
    'GIS_ACRES': ['sum', 'mean']
}).reset_index()
season_stats.columns = ['Fire Season', 'Fire Count', 'Total Acres', 'Mean Acres']

# Calculate percentages
total_fires = season_stats['Fire Count'].sum()
total_acres = season_stats['Total Acres'].sum()
season_stats['Fire %'] = season_stats['Fire Count'] / total_fires * 100
season_stats['Acres %'] = season_stats['Total Acres'] / total_acres * 100

# Order seasons logically
season_order = ['High Risk Season', 'Transition Season', 'Low Risk Season']
season_stats['sort_order'] = season_stats['Fire Season'].map({s: i for i, s in enumerate(season_order)})
season_stats = season_stats.sort_values('sort_order')

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

x_pos = np.arange(3)
season_colors = [SEASON_COLORS[s] for s in season_order]

# Panel 1: Fire Count Percentage
ax1 = axes[0]
bars1 = ax1.bar(x_pos, season_stats['Fire %'], color=season_colors, edgecolor='white')
ax1.set_xticks(x_pos)
ax1.set_xticklabels(['High Risk\n(Jun-Sep)', 'Transition\n(Oct-Jan)', 'Low Risk\n(Feb-May)'])
ax1.set_ylabel('Percentage of Total Fires', fontsize=12)
ax1.set_title('Fire Count by Season', fontsize=14, fontweight='bold')
ax1.set_ylim(0, 100)
for i, v in enumerate(season_stats['Fire %']):
    ax1.text(i, v + 2, f'{v:.1f}%', ha='center', fontsize=12, fontweight='bold')

# Panel 2: Burned Area Percentage  
ax2 = axes[1]
bars2 = ax2.bar(x_pos, season_stats['Acres %'], color=season_colors, edgecolor='white')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(['High Risk\n(Jun-Sep)', 'Transition\n(Oct-Jan)', 'Low Risk\n(Feb-May)'])
ax2.set_ylabel('Percentage of Total Burned Area', fontsize=12)
ax2.set_title('Burned Area by Season', fontsize=14, fontweight='bold')
ax2.set_ylim(0, 100)
for i, v in enumerate(season_stats['Acres %']):
    ax2.text(i, v + 2, f'{v:.1f}%', ha='center', fontsize=12, fontweight='bold')

# Panel 3: Mean Fire Size
ax3 = axes[2]
bars3 = ax3.bar(x_pos, season_stats['Mean Acres'], color=season_colors, edgecolor='white')
ax3.set_xticks(x_pos)
ax3.set_xticklabels(['High Risk\n(Jun-Sep)', 'Transition\n(Oct-Jan)', 'Low Risk\n(Feb-May)'])
ax3.set_ylabel('Average Acres per Fire', fontsize=12)
ax3.set_title('Average Fire Size by Season', fontsize=14, fontweight='bold')
for i, v in enumerate(season_stats['Mean Acres']):
    ax3.text(i, v + 100, f'{int(v):,}', ha='center', fontsize=12, fontweight='bold')

plt.suptitle(f'Fire Season Comparison\nData: {min_year_hq}-{max_year_hq} (High-Quality GPS Era)', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()

save_path = FIGURES_DIR / 'comprehensive' / '09_season_comparison.png'
fig.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print(f"Figure saved: {save_path}")

# Print season statistics
print(f"\n=== Fire Season Statistics ({min_year_hq}-{max_year_hq}) ===")
print(season_stats[['Fire Season', 'Fire Count', 'Fire %', 'Total Acres', 'Acres %', 'Mean Acres']].to_string(index=False))

## 4.5 Key Takeaways: Seasonal Patterns

**What the data tells us:**

1. **High Risk Season (Jun-Sep) dominates** - About 60% of fires and 70%+ of burned area
2. **Transition Season (Oct-Jan) is dangerous** - Santa Ana winds cause large fires despite fewer starts
3. **Low Risk Season (Feb-May) has reduced activity** - But fires still occur year-round
4. **Seasonal patterns are consistent** - The heatmap shows clear annual cycles

**ML Model Implications:**
- Month is a critical feature - strong seasonal signal
- Consider creating binary features for High Risk Season
- Fire season should be part of any prediction model
- Seasonal interactions with other variables (temperature, precipitation) are important

---
# Part 5: Fire Size Analysis

**Question: "Do all fires matter equally?"**

Fire size distribution in California follows a power law - many small fires, few large fires. But the few large fires cause the majority of damage. This is known as the **Pareto Principle** or "80/20 rule".

## 5.1 Fire Size Distribution

Most fires are small, but the distribution has a very long tail. Viewing it on a log scale reveals the true pattern.

In [ ]:
# Figure 5.1: Fire Size Distribution (Dual Panel: Raw vs Log)
# Data: HIGH-QUALITY ONLY (1993-Present)
# Outlier handling: Remove extreme outliers for cleaner visualization

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Filter to valid fire sizes
valid_sizes = gdf_hq[gdf_hq['GIS_ACRES'] > 0]['GIS_ACRES']

# Calculate percentiles
pct_99 = valid_sizes.quantile(0.99)
pct_95 = valid_sizes.quantile(0.95)

# Left panel: Raw distribution - focus on typical fires (< 10,000 acres)
# This shows the vast majority of fires more clearly
ax1 = axes[0]
typical_fires = valid_sizes[valid_sizes < 10000]  # Focus on fires under 10K acres
n_typical = len(typical_fires)
n_excluded = len(valid_sizes) - n_typical

ax1.hist(typical_fires, bins=100, color='#E74C3C', edgecolor='white', alpha=0.8)
ax1.set_xlabel('Fire Size (acres)', fontsize=12)
ax1.set_ylabel('Number of Fires', fontsize=12)
ax1.set_title(f'Fire Size Distribution (Raw Scale)\nShowing {n_typical:,} fires < 10,000 acres ({n_excluded:,} larger fires excluded)', 
              fontsize=12, fontweight='bold')
ax1.axvline(x=typical_fires.median(), color='blue', linestyle='--', linewidth=2, 
            label=f'Median: {typical_fires.median():.0f} acres')
ax1.axvline(x=1000, color='orange', linestyle='--', linewidth=2, alpha=0.7,
            label='1,000 acres threshold')
ax1.legend(fontsize=9)

# Right: Log-transformed distribution (shows ALL data including mega-fires)
ax2 = axes[1]
log_sizes = np.log10(valid_sizes)
ax2.hist(log_sizes, bins=50, color='#3498DB', edgecolor='white', alpha=0.8)
ax2.set_xlabel('Fire Size (log₁₀ acres)', fontsize=12)
ax2.set_ylabel('Number of Fires', fontsize=12)
ax2.set_title(f'Fire Size Distribution (Log Scale)\nAll {len(valid_sizes):,} fires shown', fontsize=12, fontweight='bold')

# Add vertical lines for size categories
size_thresholds = [100, 1000, 10000, 100000]
labels = ['Small (100+)', 'Medium (1K+)', 'Large (10K+)', 'Mega (100K+)']
line_colors = ['green', 'gold', 'orange', 'red']
for thresh, label, color in zip(size_thresholds, labels, line_colors):
    ax2.axvline(x=np.log10(thresh), color=color, linestyle='--', linewidth=2, alpha=0.7, 
                label=label)
ax2.legend(loc='upper left', fontsize=9)

plt.suptitle(f'Fire Size Distribution Analysis\nData: {min_year_hq}-{max_year_hq} (High-Quality GPS Era)', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()

save_path = FIGURES_DIR / 'comprehensive' / '10_fire_size_distribution.png'
fig.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print(f"Figure saved: {save_path}")

# Statistics
print(f"\n=== Fire Size Statistics ({min_year_hq}-{max_year_hq}) ===")
print(f"Total fires: {len(valid_sizes):,}")
print(f"Mean size: {valid_sizes.mean():,.0f} acres")
print(f"Median size: {valid_sizes.median():,.0f} acres")
print(f"95th percentile: {pct_95:,.0f} acres")
print(f"99th percentile: {pct_99:,.0f} acres")
print(f"Max size: {valid_sizes.max():,.0f} acres")
print(f"\nFires by size category:")
print(f"  < 100 acres: {(valid_sizes < 100).sum():,} ({(valid_sizes < 100).sum()/len(valid_sizes)*100:.1f}%)")
print(f"  100-1K acres: {((valid_sizes >= 100) & (valid_sizes < 1000)).sum():,}")
print(f"  1K-10K acres: {((valid_sizes >= 1000) & (valid_sizes < 10000)).sum():,}")
print(f"  10K-100K acres: {((valid_sizes >= 10000) & (valid_sizes < 100000)).sum():,}")
print(f"  > 100K acres (mega): {(valid_sizes >= 100000).sum():,}")

## 5.2 The Pareto Principle (80/20 Rule)

In wildfires, a small percentage of fires cause the majority of damage. This is critical for resource allocation and risk modeling.

In [ ]:
# Figure 5.2: Pareto Analysis - Lorenz Curve and Top Fires
# Data: HIGH-QUALITY ONLY (1993-Present)

fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Sort fires by size (largest first)
sorted_fires = gdf_hq.sort_values('GIS_ACRES', ascending=False).reset_index(drop=True)
sorted_fires['cumulative_acres'] = sorted_fires['GIS_ACRES'].cumsum()
sorted_fires['cumulative_pct'] = sorted_fires['cumulative_acres'] / sorted_fires['GIS_ACRES'].sum() * 100
sorted_fires['fire_pct'] = (sorted_fires.index + 1) / len(sorted_fires) * 100

# Left: Lorenz Curve
ax1 = axes[0]
ax1.plot(sorted_fires['fire_pct'], sorted_fires['cumulative_pct'], 'b-', linewidth=2.5, label='Actual Distribution')
ax1.plot([0, 100], [0, 100], 'k--', linewidth=1, alpha=0.5, label='Perfect Equality')
ax1.fill_between(sorted_fires['fire_pct'], sorted_fires['cumulative_pct'], color='#E74C3C', alpha=0.3)

# Find 80% point
idx_80 = (sorted_fires['cumulative_pct'] >= 80).idxmax()
fire_pct_at_80 = sorted_fires.loc[idx_80, 'fire_pct']
ax1.axhline(y=80, color='red', linestyle='--', alpha=0.7)
ax1.axvline(x=fire_pct_at_80, color='red', linestyle='--', alpha=0.7)
ax1.scatter([fire_pct_at_80], [80], s=100, color='red', zorder=5)
ax1.annotate(f'{fire_pct_at_80:.1f}% of fires\ncause 80% of burned area', 
             xy=(fire_pct_at_80, 80), xytext=(fire_pct_at_80 + 20, 60),
             fontsize=11, fontweight='bold', color='red',
             arrowprops=dict(arrowstyle='->', color='red'))

ax1.set_xlabel('Cumulative % of Fires (largest first)', fontsize=12)
ax1.set_ylabel('Cumulative % of Burned Area', fontsize=12)
ax1.set_title(f'Fire Size Inequality: Lorenz Curve\nData: {min_year_hq}-{max_year_hq}', fontsize=14, fontweight='bold')
ax1.legend(loc='lower right')
ax1.set_xlim(0, 100)
ax1.set_ylim(0, 100)

# Right: Top 10 fires contribution
ax2 = axes[1]
top_10 = sorted_fires.head(10)
total_acres = gdf_hq['GIS_ACRES'].sum()
top_10_pct = top_10['GIS_ACRES'].sum() / total_acres * 100

y_pos = np.arange(10)
bars = ax2.barh(y_pos, top_10['GIS_ACRES'] / 1e6, color='#E74C3C', edgecolor='white')
ax2.set_yticks(y_pos)
fire_labels = [f"{row['FIRE_NAME']} ({int(row['YEAR_'])})" if pd.notna(row['FIRE_NAME']) 
               else f"Unnamed ({int(row['YEAR_'])})" for _, row in top_10.iterrows()]
ax2.set_yticklabels(fire_labels)
ax2.set_xlabel('Million Acres Burned', fontsize=12)
ax2.set_title(f'Top 10 Largest Fires ({min_year_hq}-{max_year_hq})\n{top_10_pct:.1f}% of total burned area', 
              fontsize=14, fontweight='bold')
ax2.invert_yaxis()

for i, v in enumerate(top_10['GIS_ACRES'] / 1e6):
    ax2.text(v + 0.02, i, f'{v:.2f}M', va='center', fontsize=10)

plt.suptitle(f'The Pareto Principle: A Few Mega-Fires Dominate\nData: {min_year_hq}-{max_year_hq} (High-Quality)', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()

save_path = FIGURES_DIR / 'comprehensive' / '11_pareto_analysis.png'
fig.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print(f"Figure saved: {save_path}")

# Pareto statistics
print(f"\n=== Pareto Statistics ({min_year_hq}-{max_year_hq}) ===")
print(f"Top 1% of fires cause: {sorted_fires.loc[int(len(sorted_fires)*0.01), 'cumulative_pct']:.1f}% of burned area")
print(f"Top 5% of fires cause: {sorted_fires.loc[int(len(sorted_fires)*0.05), 'cumulative_pct']:.1f}% of burned area")
print(f"Top 10% of fires cause: {sorted_fires.loc[int(len(sorted_fires)*0.10), 'cumulative_pct']:.1f}% of burned area")
print(f"Top 10 fires cause: {top_10_pct:.1f}% of burned area")

## 5.3 Fire Size Categories

Let's break down fires into size categories and see how each contributes to total burned area.

In [ ]:
# Figure 5.3: Fire Size Categories (Dual Panel)
# Data: HIGH-QUALITY ONLY (1993-Present)

size_order = ['Very Small (<100)', 'Small (100-1K)', 'Medium (1K-10K)', 'Large (10K-100K)', 'Mega (100K+)']
size_colors = ['#27AE60', '#F1C40F', '#E67E22', '#E74C3C', '#8E44AD']

# Calculate statistics by size category
size_stats = gdf_hq.groupby('SIZE_CATEGORY').agg({
    'OBJECTID': 'count',
    'GIS_ACRES': 'sum'
}).rename(columns={'OBJECTID': 'count', 'GIS_ACRES': 'total_acres'}).reset_index()

# Reorder
size_stats['sort_order'] = size_stats['SIZE_CATEGORY'].map({s: i for i, s in enumerate(size_order)})
size_stats = size_stats.sort_values('sort_order')

# Calculate percentages
size_stats['count_pct'] = size_stats['count'] / size_stats['count'].sum() * 100
size_stats['acres_pct'] = size_stats['total_acres'] / size_stats['total_acres'].sum() * 100

fig, axes = plt.subplots(1, 2, figsize=(16, 7))

x_pos = np.arange(len(size_order))

# Left: Fire Count by Category
ax1 = axes[0]
bars1 = ax1.bar(x_pos, size_stats['count_pct'], color=size_colors, edgecolor='white')
ax1.set_xticks(x_pos)
ax1.set_xticklabels(size_order, rotation=30, ha='right')
ax1.set_ylabel('Percentage of Total Fires', fontsize=12)
ax1.set_title('Fire Count by Size Category\nMost fires are small', fontsize=14, fontweight='bold')
for i, v in enumerate(size_stats['count_pct']):
    ax1.text(i, v + 1, f'{v:.1f}%', ha='center', fontsize=10, fontweight='bold')

# Right: Burned Area by Category
ax2 = axes[1]
bars2 = ax2.bar(x_pos, size_stats['acres_pct'], color=size_colors, edgecolor='white')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(size_order, rotation=30, ha='right')
ax2.set_ylabel('Percentage of Total Burned Area', fontsize=12)
ax2.set_title('Burned Area by Size Category\nMega-fires dominate destruction', fontsize=14, fontweight='bold')
for i, v in enumerate(size_stats['acres_pct']):
    ax2.text(i, v + 1, f'{v:.1f}%', ha='center', fontsize=10, fontweight='bold')

plt.suptitle(f'The Size Paradox: Many Small Fires vs Few Mega-Fires\nData: {min_year_hq}-{max_year_hq} (High-Quality)', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()

save_path = FIGURES_DIR / 'comprehensive' / '12_fire_size_categories.png'
fig.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print(f"Figure saved: {save_path}")

print(f"\n=== Fire Size Category Statistics ({min_year_hq}-{max_year_hq}) ===")
print(size_stats[['SIZE_CATEGORY', 'count', 'count_pct', 'total_acres', 'acres_pct']].to_string(index=False))

## 5.4 Key Takeaways: Fire Size Analysis

**What the data tells us:**

1. **Fire size follows a power law** - Many small fires, few large fires
2. **The 80/20 rule applies** - ~5% of fires cause ~80% of burned area
3. **Mega-fires dominate damage** - While rare, they determine fire season severity
4. **The top 10 fires matter enormously** - They account for a significant portion of total burned area

**ML Model Implications:**
- Predicting mega-fires is more important than predicting small fires
- Consider log-transforming fire size for modeling
- Class imbalance: most grid cells never burn in a mega-fire
- May need separate models for "fire occurrence" vs "fire size"

---
# Part 6: Spatial Analysis

**Question: "Where do fires burn most frequently in California?"**

This section analyzes the geographic patterns of fire occurrence across California. The final visualization shows cumulative fire risk - areas that have burned multiple times over the past 30+ years.

## 6.1 California Fire Overview Map

A map of all fire perimeters (1993+) shows the spatial distribution of fires across the state.

In [ ]:
# Figure 6.1: California Fire Overview Map
# Data: HIGH-QUALITY ONLY (1993-Present)

fig, ax = plt.subplots(figsize=(12, 16))

# Get bounds for consistent display
if ca_boundary is not None:
    xmin, ymin, xmax, ymax = ca_boundary.total_bounds
else:
    xmin, ymin, xmax, ymax = gdf_hq.total_bounds

ax.set_xlim(xmin - 50000, xmax + 50000)
ax.set_ylim(ymin - 50000, ymax + 50000)

# Plot California boundary first
if ca_boundary is not None:
    ca_boundary.plot(ax=ax, facecolor='#f5f5f5', edgecolor='black', linewidth=2, alpha=0.8)

# Plot all fire perimeters colored by decade
decade_cmap = plt.cm.YlOrRd
decades_present = sorted(gdf_hq['DECADE'].unique())
norm = Normalize(vmin=min(decades_present), vmax=max(decades_present))

for decade in decades_present:
    decade_fires = gdf_hq[gdf_hq['DECADE'] == decade]
    color = decade_cmap(norm(decade))
    decade_fires.plot(ax=ax, facecolor=color, edgecolor='none', alpha=0.6)

try:
    ctx.add_basemap(ax, crs=gdf_hq.crs.to_string(), source=ctx.providers.CartoDB.Positron, alpha=0.3)
except:
    pass

# Colorbar for decades
sm = plt.cm.ScalarMappable(cmap=decade_cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, shrink=0.5, label='Decade')
cbar.set_ticks(decades_present)
cbar.set_ticklabels([f'{int(d)}s' for d in decades_present])

ax.set_title(f'California Fire Perimeters by Decade\nData: {min_year_hq}-{max_year_hq} (High-Quality GPS Era)', 
             fontsize=16, fontweight='bold')
ax.set_xlabel('Easting (meters)', fontsize=12)
ax.set_ylabel('Northing (meters)', fontsize=12)
ax.set_aspect('equal')

plt.tight_layout()

save_path = FIGURES_DIR / 'comprehensive' / '13_spatial_overview.png'
fig.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print(f"Figure saved: {save_path}")

## 6.2 Interactive Map: Mega-Fires

An interactive map allows exploration of the largest fires. Click on fire perimeters for details.

In [ ]:
# Figure 6.2: Interactive Folium Map of Mega-Fires
# Get mega-fires (>100K acres)
mega_fires = gdf_hq[gdf_hq['GIS_ACRES'] >= 100000].copy()
mega_fires_wgs84 = mega_fires.to_crs(CRS_WGS84)

print(f"Creating interactive map with {len(mega_fires)} mega-fires...")

# Center on California
ca_center = [37.0, -119.5]
m = folium.Map(location=ca_center, zoom_start=6, tiles='CartoDB positron')

# Add each mega-fire as a polygon
for idx, row in mega_fires_wgs84.iterrows():
    popup_text = f"""
    <b>{row['FIRE_NAME'] if pd.notna(row['FIRE_NAME']) else 'Unnamed'}</b><br>
    Year: {int(row['YEAR_'])}<br>
    Size: {row['GIS_ACRES']:,.0f} acres<br>
    Agency: {row.get('AGENCY_LABEL', row.get('AGENCY', 'Unknown'))}<br>
    Cause: {row.get('CAUSE_LABEL', row.get('CAUSE', 'Unknown'))}
    """
    
    # Color by year (more recent = more red)
    year_norm = (row['YEAR_'] - 1993) / (2025 - 1993)
    color = plt.cm.YlOrRd(year_norm)
    hex_color = mcolors.rgb2hex(color)
    
    folium.GeoJson(
        row.geometry.__geo_interface__,
        style_function=lambda x, color=hex_color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.6
        },
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(m)

# Save and display
map_path = INTERACTIVE_DIR / 'mega_fires_map.html'
m.save(str(map_path))
print(f"Interactive map saved: {map_path}")

# Display the map
display(m)

## 6.3 Cumulative Fire Risk Map

**The Most Important Visualization**

This map shows the cumulative "burn frequency" across California - areas that have burned multiple times since 1993. This is the foundation for understanding spatial fire risk:

- **White/Bright areas**: Burned multiple times = HIGH RISK
- **Red/Orange areas**: Burned once or twice = MODERATE RISK  
- **Dark areas**: Never burned (in our record) = LOWER RECENT RISK

This visualization directly supports the building of our spatial fire prediction model.

In [ ]:
# Figure 6.3: Cumulative Fire Risk Map with Basemap
# Data: HIGH-QUALITY ONLY (1993-Present)
# Displayed on terrain basemap for geographic context

import contextily as ctx

print("Creating cumulative fire risk map with terrain basemap...")
print("This may take a few minutes...")

# Reproject fire data to Web Mercator (EPSG:3857) for basemap compatibility
gdf_hq_web = gdf_hq.to_crs(epsg=3857)
valid_fires_web = gdf_hq_web[gdf_hq_web.geometry.is_valid & ~gdf_hq_web.geometry.is_empty]

# Get bounds in Web Mercator
xmin, ymin, xmax, ymax = valid_fires_web.total_bounds

# Add some padding
padding = 50000  # meters
xmin -= padding
ymin -= padding
xmax += padding
ymax += padding

# Create grid at 1km resolution
resolution = 1000  # meters
width = int(np.ceil((xmax - xmin) / resolution))
height = int(np.ceil((ymax - ymin) / resolution))

print(f"Grid dimensions: {width} x {height} cells ({resolution}m resolution)")

# Create proper affine transform
from rasterio.transform import Affine
transform = Affine(resolution, 0, xmin, 0, -resolution, ymax)

burn_frequency = np.zeros((height, width), dtype=np.float32)

print(f"Rasterizing {len(valid_fires_web):,} fire perimeters...")

for idx, fire in valid_fires_web.iterrows():
    try:
        fire_raster = rasterize(
            [(fire.geometry, 1)],
            out_shape=(height, width),
            transform=transform,
            fill=0,
            dtype='uint8'
        )
        burn_frequency += fire_raster
    except:
        continue

print(f"Max burn frequency: {burn_frequency.max():.0f} times burned")
print(f"Cells burned at least once: {(burn_frequency > 0).sum():,}")

# Create figure
fig, ax = plt.subplots(figsize=(12, 14))

# Set extent first
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

# Add basemap FIRST (underneath everything)
try:
    ctx.add_basemap(ax, 
                    crs='EPSG:3857',
                    source=ctx.providers.CartoDB.Positron,  # Clean, light basemap
                    zoom=7,
                    alpha=1.0)
    print("Added CartoDB Positron basemap")
except Exception as e:
    print(f"Basemap error: {e}")
    # Fallback to simple background
    ax.set_facecolor('#f5f5f5')

# Create custom colormap: transparent -> yellow -> orange -> red -> white
from matplotlib.colors import LinearSegmentedColormap
colors_fire = ['#000000', '#8B0000', '#FF4500', '#FF8C00', '#FFD700', '#FFFF00', '#FFFFFF']
fire_cmap = LinearSegmentedColormap.from_list('fire_intensity', colors_fire, N=256)

# Mask zero values for transparency
burn_masked = np.ma.masked_where(burn_frequency == 0, burn_frequency)

# Plot burn frequency on top of basemap
im = ax.imshow(burn_masked,
               cmap=fire_cmap,
               extent=[xmin, xmax, ymin, ymax],
               origin='upper',
               vmin=0,
               vmax=max(burn_frequency.max(), 1),
               alpha=0.75,
               interpolation='nearest',
               zorder=2)

# Add subtle California boundary outline (thin, doesn't hide data)
if ca_boundary is not None:
    ca_web = ca_boundary.to_crs(epsg=3857)
    ca_web.boundary.plot(ax=ax, color='#333333', linewidth=0.8, alpha=0.5, zorder=3)

# Add colorbar
cbar = plt.colorbar(im, ax=ax, shrink=0.6, pad=0.02)
cbar.set_label('Burn Frequency (times burned since 1993)', fontsize=11)
cbar.ax.tick_params(labelsize=10)

# Title
ax.set_title(f'Cumulative Fire Risk Map: California {min_year_hq}-{max_year_hq}\n'
             f'30+ Years of Fire History | Brighter = More Frequent Burning', 
             fontsize=14, fontweight='bold')

# Clean up axes
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_xticks([])
ax.set_yticks([])

# Legend box
legend_text = (
    f'🔥 BURN FREQUENCY\n'
    f'━━━━━━━━━━━━━━━━━━━━\n'
    f'WHITE/YELLOW: 3+ burns\n'
    f'ORANGE/RED: 1-2 burns\n'
    f'TRANSPARENT: No burns\n'
    f'━━━━━━━━━━━━━━━━━━━━\n'
    f'Period: {min_year_hq}-{max_year_hq}\n'
    f'Resolution: {resolution}m grid\n'
    f'Max burns: {burn_frequency.max():.0f}×'
)
ax.text(0.02, 0.98, legend_text,
        transform=ax.transAxes, fontsize=10, verticalalignment='top',
        fontfamily='monospace',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.9),
        zorder=10)

# Attribution
ax.text(0.99, 0.01, 'Basemap: CartoDB Positron | Data: CAL FIRE FRAP',
        transform=ax.transAxes, fontsize=8, ha='right', va='bottom',
        color='gray', alpha=0.7)

plt.tight_layout()

save_path = FIGURES_DIR / 'comprehensive' / '14_cumulative_fire_risk.png'
fig.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print(f"\nFigure saved: {save_path}")
print(f"\n⭐ This map shows where fires have burned repeatedly since 1993.")
print(f"   Bright areas indicate high fire recurrence - key for risk modeling.")

## 6.4 Key Takeaways: Spatial Patterns

**What the cumulative map tells us:**

1. **Some areas burn repeatedly** - The brightest spots have burned 3+ times in 30 years
2. **Northern California and Sierra foothills** - Highest burn frequency
3. **Southern California has frequent fires** - Many overlapping perimeters
4. **Coastal areas burn less frequently** - Marine influence moderates fire risk
5. **The Central Valley rarely burns** - Agricultural land has different fire dynamics

**ML Model Implications:**
- Historical burn frequency is a strong predictor of future fires
- Spatial features (location, elevation, vegetation) are critical
- Grid-based modeling allows direct use of burn frequency as a feature
- Phase 2 will create 800m × 800m grid cells aligned with this concept

---
# Part 7: Fire Causes

**Question: "What causes California wildfires?"**

Understanding fire causes is critical for prevention strategies and resource allocation. However, determining the cause of a wildfire is extremely challenging - investigators must work in hazardous post-fire environments, and evidence is often destroyed by the fire itself.

**Key insight:** The largest category of fire causes is **Unknown/Unidentified**, reflecting the inherent difficulty of fire cause investigation. Among fires with known causes, both lightning (natural) and human activities contribute significantly.

In [ ]:
# Figure 7.1: Fire Causes Analysis (Dual Panel)
# Data: HIGH-QUALITY ONLY (1993-Present)

cause_stats = gdf_hq.groupby('CAUSE_LABEL').agg({
    'OBJECTID': 'count',
    'GIS_ACRES': 'sum'
}).rename(columns={'OBJECTID': 'count', 'GIS_ACRES': 'acres'}).reset_index()

cause_stats = cause_stats.sort_values('count', ascending=False)
cause_stats['count_pct'] = cause_stats['count'] / cause_stats['count'].sum() * 100
cause_stats['acres_pct'] = cause_stats['acres'] / cause_stats['acres'].sum() * 100

top_causes = cause_stats.head(10)

cause_colors = {
    'Lightning': '#F39C12',
    'Unknown/Unidentified': '#95A5A6',
    'Equipment Use': '#3498DB',
    'Debris Burning': '#2ECC71',
    'Arson': '#E74C3C',
    'Miscellaneous': '#9B59B6',
    'Vehicle': '#1ABC9C',
    'Campfire': '#E67E22',
    'Powerline': '#8E44AD',
    'Playing with Fire': '#F1C40F'
}

fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Left: Fire Count by Cause
ax1 = axes[0]
colors = [cause_colors.get(c, '#95A5A6') for c in top_causes['CAUSE_LABEL']]
bars1 = ax1.barh(range(len(top_causes)), top_causes['count'], color=colors, edgecolor='white')
ax1.set_yticks(range(len(top_causes)))
ax1.set_yticklabels(top_causes['CAUSE_LABEL'])
ax1.set_xlabel('Number of Fires', fontsize=12)
ax1.set_title(f'Fire Count by Cause\nData: {min_year_hq}-{max_year_hq}', fontsize=14, fontweight='bold')
ax1.invert_yaxis()
for i, (v, pct) in enumerate(zip(top_causes['count'], top_causes['count_pct'])):
    ax1.text(v + 50, i, f'{int(v):,} ({pct:.1f}%)', va='center', fontsize=9)

# Right: Burned Area by Cause
ax2 = axes[1]
cause_stats_by_acres = cause_stats.sort_values('acres', ascending=False).head(10)
colors2 = [cause_colors.get(c, '#95A5A6') for c in cause_stats_by_acres['CAUSE_LABEL']]
bars2 = ax2.barh(range(len(cause_stats_by_acres)), cause_stats_by_acres['acres']/1e6, color=colors2, edgecolor='white')
ax2.set_yticks(range(len(cause_stats_by_acres)))
ax2.set_yticklabels(cause_stats_by_acres['CAUSE_LABEL'])
ax2.set_xlabel('Million Acres Burned', fontsize=12)
ax2.set_title(f'Burned Area by Cause\nData: {min_year_hq}-{max_year_hq}', fontsize=14, fontweight='bold')
ax2.invert_yaxis()
for i, (v, pct) in enumerate(zip(cause_stats_by_acres['acres']/1e6, cause_stats_by_acres['acres_pct'])):
    ax2.text(v + 0.1, i, f'{v:.1f}M ({pct:.1f}%)', va='center', fontsize=9)

plt.suptitle(f'Fire Causes Analysis\nData: {min_year_hq}-{max_year_hq} (High-Quality GPS Era)', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()

save_path = FIGURES_DIR / 'comprehensive' / '15_fire_causes.png'
fig.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print(f"Figure saved: {save_path}")

# Human vs Natural
lightning_count = gdf_hq[gdf_hq['CAUSE_LABEL'] == 'Lightning']['OBJECTID'].count()
lightning_acres = gdf_hq[gdf_hq['CAUSE_LABEL'] == 'Lightning']['GIS_ACRES'].sum()
total_count = gdf_hq['OBJECTID'].count()
total_acres = gdf_hq['GIS_ACRES'].sum()

print(f"\n=== Human vs Natural Fires ({min_year_hq}-{max_year_hq}) ===")
print(f"Lightning fires: {lightning_count:,} ({lightning_count/total_count*100:.1f}%)")
print(f"Human-caused fires: {total_count - lightning_count:,} ({(total_count-lightning_count)/total_count*100:.1f}%)")
print(f"\nLightning burned area: {lightning_acres/1e6:.2f}M acres ({lightning_acres/total_acres*100:.1f}%)")
print(f"Human-caused burned area: {(total_acres-lightning_acres)/1e6:.2f}M acres ({(total_acres-lightning_acres)/total_acres*100:.1f}%)")

---
# Part 8: Agency Response

**Question: "Who responds to California wildfires?"**

Multiple agencies manage California's wildlands, each responsible for different jurisdictions.

In [ ]:
# Figure 8.1: Agency Response Analysis (Dual Panel)
# Data: 1993-Present (High-Quality GPS Era)
max_year = gdf_hq['YEAR_'].max()

# Get agency statistics
agency_stats = gdf_hq.groupby('AGENCY_LABEL').agg({
    'OBJECTID': 'count',
    'GIS_ACRES': 'sum'
}).rename(columns={'OBJECTID': 'count', 'GIS_ACRES': 'acres'}).reset_index()

# Sort by count
agency_stats = agency_stats.sort_values('count', ascending=False)

# Calculate percentages
agency_stats['count_pct'] = agency_stats['count'] / agency_stats['count'].sum() * 100
agency_stats['acres_pct'] = agency_stats['acres'] / agency_stats['acres'].sum() * 100

# Take top agencies
top_agencies = agency_stats.head(8)

# Color scheme for agencies
agency_colors = {
    'CAL FIRE': '#E63946',
    'USDA Forest Service': '#457B9D',
    'Contract County': '#27AE60',
    'Local Responsibility Area': '#E9C46A',
    'Bureau of Land Management': '#F4A261',
    'National Park Service': '#2A9D8F',
    'California State Parks': '#1ABC9C',
    'Other': '#95A5A6'
}

fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Left: Fire Count by Agency
ax1 = axes[0]
colors = [agency_colors.get(a, '#95A5A6') for a in top_agencies['AGENCY_LABEL']]
bars1 = ax1.barh(range(len(top_agencies)), top_agencies['count'], color=colors, edgecolor='white')
ax1.set_yticks(range(len(top_agencies)))
ax1.set_yticklabels(top_agencies['AGENCY_LABEL'])
ax1.set_xlabel('Number of Fires', fontsize=12)
ax1.set_title(f'Fire Count by Responding Agency (1993-{max_year})\nWho responds to the most fires?', fontsize=14, fontweight='bold')
ax1.invert_yaxis()
for i, (v, pct) in enumerate(zip(top_agencies['count'], top_agencies['count_pct'])):
    ax1.text(v + 30, i, f'{int(v):,} ({pct:.1f}%)', va='center', fontsize=9)

# Right: Burned Area by Agency
ax2 = axes[1]
agency_stats_by_acres = agency_stats.sort_values('acres', ascending=False).head(8)
colors2 = [agency_colors.get(a, '#95A5A6') for a in agency_stats_by_acres['AGENCY_LABEL']]
bars2 = ax2.barh(range(len(agency_stats_by_acres)), agency_stats_by_acres['acres']/1e6, color=colors2, edgecolor='white')
ax2.set_yticks(range(len(agency_stats_by_acres)))
ax2.set_yticklabels(agency_stats_by_acres['AGENCY_LABEL'])
ax2.set_xlabel('Million Acres Burned', fontsize=12)
ax2.set_title(f'Burned Area by Jurisdiction (1993-{max_year})\nWhere does the most land burn?', fontsize=14, fontweight='bold')
ax2.invert_yaxis()
for i, (v, pct) in enumerate(zip(agency_stats_by_acres['acres']/1e6, agency_stats_by_acres['acres_pct'])):
    ax2.text(v + 0.1, i, f'{v:.1f}M ({pct:.1f}%)', va='center', fontsize=9)

plt.suptitle(f'Agency Jurisdiction Analysis (1993-{max_year}): State vs Federal Responsibilities\nData: High-Quality GPS Era | USDA Forest Service has the most burned area', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()

save_path = FIGURES_DIR / 'comprehensive' / '16_agency_response.png'
fig.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print(f"Figure saved: {save_path}")

## 8.2 Agency Fire Activity Over Time

How has fire activity changed for each agency? Let's look at trends in fire counts and burned acres by agency over the 1993-present period.

**Agency Acronyms:**
| Code | Agency Name |
|------|-------------|
| CDF | CAL FIRE (California Dept of Forestry & Fire Protection) |
| USF | USDA Forest Service |
| BLM | Bureau of Land Management |
| NPS | National Park Service |
| LRA | Local Responsibility Area |
| CCO | Contract County Organization |
| BIA | Bureau of Indian Affairs |
| FWS | US Fish and Wildlife Service |
| DOD | Department of Defense |

In [ ]:
# Figure 8.2: Agency Fire Activity Over Time (1993-Present)

# Get top 6 agencies by total fire count
top_agencies = gdf_hq.groupby('AGENCY').size().nlargest(6).index.tolist()

# Agency labels for legend
agency_labels = {
    'CDF': 'CAL FIRE (State)',
    'USF': 'USDA Forest Service (Federal)',
    'BLM': 'Bureau of Land Management',
    'NPS': 'National Park Service',
    'LRA': 'Local Responsibility Area',
    'CCO': 'Contract County',
    'BIA': 'Bureau of Indian Affairs',
    'FWS': 'US Fish & Wildlife',
    'DOD': 'Dept of Defense'
}

# Colors for agencies
agency_colors = {
    'CDF': '#E63946',  # Red
    'USF': '#457B9D',  # Blue
    'BLM': '#F4A261',  # Orange
    'NPS': '#2A9D8F',  # Teal
    'LRA': '#E9C46A',  # Yellow
    'CCO': '#27AE60',  # Green
}

# Aggregate by year and agency
agency_yearly = gdf_hq[gdf_hq['AGENCY'].isin(top_agencies)].groupby(['YEAR_', 'AGENCY']).agg({
    'OBJECTID': 'count',
    'GIS_ACRES': 'sum'
}).rename(columns={'OBJECTID': 'fire_count', 'GIS_ACRES': 'acres_burned'}).reset_index()

fig, axes = plt.subplots(2, 1, figsize=(16, 12))

# Top: Fire Count by Agency Over Time
ax1 = axes[0]
for agency in top_agencies:
    data = agency_yearly[agency_yearly['AGENCY'] == agency]
    label = agency_labels.get(agency, agency)
    color = agency_colors.get(agency, '#95A5A6')
    ax1.plot(data['YEAR_'], data['fire_count'], 
             label=label, color=color, linewidth=2, marker='o', markersize=3)

ax1.set_xlabel('Year', fontsize=12)
ax1.set_ylabel('Number of Fires', fontsize=12)
ax1.set_title(f'Fire Count by Agency Over Time\nData: {min_year_hq}-{max_year_hq}', fontsize=14, fontweight='bold')
ax1.legend(loc='upper left', fontsize=10, title='Agency')
ax1.grid(True, alpha=0.3)
ax1.set_xlim(min_year_hq, max_year_hq)

# Bottom: Burned Acres by Agency Over Time
ax2 = axes[1]
for agency in top_agencies:
    data = agency_yearly[agency_yearly['AGENCY'] == agency]
    label = agency_labels.get(agency, agency)
    color = agency_colors.get(agency, '#95A5A6')
    ax2.plot(data['YEAR_'], data['acres_burned'] / 1e6, 
             label=label, color=color, linewidth=2, marker='o', markersize=3)

ax2.set_xlabel('Year', fontsize=12)
ax2.set_ylabel('Million Acres Burned', fontsize=12)
ax2.set_title(f'Burned Acres by Agency Over Time\nData: {min_year_hq}-{max_year_hq}', fontsize=14, fontweight='bold')
ax2.legend(loc='upper left', fontsize=10, title='Agency')
ax2.grid(True, alpha=0.3)
ax2.set_xlim(min_year_hq, max_year_hq)

plt.suptitle(f'Agency Fire Activity Trends ({min_year_hq}-{max_year_hq})', fontsize=16, fontweight='bold', y=1.01)
plt.tight_layout()

save_path = FIGURES_DIR / 'comprehensive' / '17_agency_trends_over_time.png'
fig.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print(f"Figure saved: {save_path}")

# Summary table
print(f"\n=== Agency Summary ({min_year_hq}-{max_year_hq}) ===")
agency_summary = gdf_hq.groupby('AGENCY').agg({
    'OBJECTID': 'count',
    'GIS_ACRES': 'sum'
}).rename(columns={'OBJECTID': 'Fire Count', 'GIS_ACRES': 'Total Acres'})
agency_summary['Avg Size'] = agency_summary['Total Acres'] / agency_summary['Fire Count']
agency_summary = agency_summary.sort_values('Fire Count', ascending=False)
agency_summary['Agency Name'] = agency_summary.index.map(lambda x: agency_labels.get(x, x))
print(agency_summary[['Agency Name', 'Fire Count', 'Total Acres', 'Avg Size']].head(10).to_string())

## 8.3 Fire Activity by Unit ID

CAL FIRE divides California into administrative units. Let's see which units have the most fire activity.

**Common Unit ID Codes:**
| Code | Unit Name | Region |
|------|-----------|--------|
| SHU | Shasta-Trinity Unit | Northern CA |
| TUU | Tuolumne-Calaveras Unit | Sierra Nevada |
| NEU | Nevada-Yuba-Placer Unit | Sierra Foothills |
| BTU | Butte Unit | Northern CA |
| LNU | Sonoma-Lake-Napa Unit | North Bay |
| SCU | Santa Clara Unit | Bay Area |
| RRU | Riverside Unit | Southern CA |
| MVU | San Diego Unit | Southern CA |
| LAC | Los Angeles County | Southern CA |
| ORC | Orange County | Southern CA |

In [ ]:
# Figure 8.3: Fire Activity by Unit ID (1993-Present)

# Unit name mappings (expand as needed)
unit_labels = {
    'SHU': 'Shasta-Trinity',
    'TUU': 'Tuolumne-Calaveras', 
    'NEU': 'Nevada-Yuba-Placer',
    'BTU': 'Butte',
    'LNU': 'Sonoma-Lake-Napa',
    'SCU': 'Santa Clara',
    'RRU': 'Riverside',
    'MVU': 'San Diego (Mt View)',
    'LAC': 'Los Angeles County',
    'ORC': 'Orange County',
    'SLU': 'San Luis Obispo',
    'TCU': 'Tehama-Glenn',
    'AEU': 'Amador-El Dorado',
    'FKU': 'Fresno-Kings',
    'MEU': 'Mendocino',
    'HUU': 'Humboldt-Del Norte',
    'KRN': 'Kern County',
    'VNC': 'Ventura County',
    'SBC': 'Santa Barbara County',
    'MRN': 'Marin County'
}

# Get top 15 units by fire count
unit_stats = gdf_hq.groupby('UNIT_ID').agg({
    'OBJECTID': 'count',
    'GIS_ACRES': 'sum'
}).rename(columns={'OBJECTID': 'fire_count', 'GIS_ACRES': 'acres_burned'})
unit_stats = unit_stats.sort_values('fire_count', ascending=False)
top_units = unit_stats.head(15)

# Add unit labels
top_units['unit_name'] = top_units.index.map(lambda x: unit_labels.get(x, x))
top_units['display_name'] = top_units.index + ' - ' + top_units['unit_name']

fig, axes = plt.subplots(1, 2, figsize=(18, 10))

# Left: Fire Count by Unit
ax1 = axes[0]
colors1 = plt.cm.Reds(np.linspace(0.3, 0.9, len(top_units)))
bars1 = ax1.barh(range(len(top_units)), top_units['fire_count'], color=colors1, edgecolor='white')
ax1.set_yticks(range(len(top_units)))
ax1.set_yticklabels(top_units['display_name'], fontsize=10)
ax1.set_xlabel('Number of Fires', fontsize=12)
ax1.set_title(f'Top 15 Units by Fire Count\nData: {min_year_hq}-{max_year_hq}', fontsize=14, fontweight='bold')
ax1.invert_yaxis()

# Add value labels
for i, v in enumerate(top_units['fire_count']):
    ax1.text(v + 20, i, f'{int(v):,}', va='center', fontsize=9)

# Right: Burned Acres by Unit (sorted by acres)
ax2 = axes[1]
top_units_acres = unit_stats.sort_values('acres_burned', ascending=False).head(15)
top_units_acres['unit_name'] = top_units_acres.index.map(lambda x: unit_labels.get(x, x))
top_units_acres['display_name'] = top_units_acres.index + ' - ' + top_units_acres['unit_name']

colors2 = plt.cm.Oranges(np.linspace(0.3, 0.9, len(top_units_acres)))
bars2 = ax2.barh(range(len(top_units_acres)), top_units_acres['acres_burned'] / 1e6, color=colors2, edgecolor='white')
ax2.set_yticks(range(len(top_units_acres)))
ax2.set_yticklabels(top_units_acres['display_name'], fontsize=10)
ax2.set_xlabel('Million Acres Burned', fontsize=12)
ax2.set_title(f'Top 15 Units by Burned Acres\nData: {min_year_hq}-{max_year_hq}', fontsize=14, fontweight='bold')
ax2.invert_yaxis()

# Add value labels
for i, v in enumerate(top_units_acres['acres_burned'] / 1e6):
    ax2.text(v + 0.05, i, f'{v:.2f}M', va='center', fontsize=9)

plt.suptitle(f'Fire Activity by CAL FIRE Unit ({min_year_hq}-{max_year_hq})', fontsize=16, fontweight='bold', y=1.01)
plt.tight_layout()

save_path = FIGURES_DIR / 'comprehensive' / '18_unit_fire_activity.png'
fig.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print(f"Figure saved: {save_path}")

# Print summary table
print(f"\n=== Top 15 Units Summary ({min_year_hq}-{max_year_hq}) ===")
summary_table = unit_stats.head(15).copy()
summary_table['Unit Name'] = summary_table.index.map(lambda x: unit_labels.get(x, x))
summary_table['Avg Fire Size'] = summary_table['acres_burned'] / summary_table['fire_count']
summary_table = summary_table[['Unit Name', 'fire_count', 'acres_burned', 'Avg Fire Size']]
summary_table.columns = ['Unit Name', 'Fire Count', 'Total Acres', 'Avg Size (acres)']
print(summary_table.to_string())

In [ ]:
# Figure 8.4: Unit Fire Activity Trends Over Time

# Get top 8 units by total fire count for trend analysis
top_8_units = gdf_hq.groupby('UNIT_ID').size().nlargest(8).index.tolist()

# Aggregate by year and unit
unit_yearly = gdf_hq[gdf_hq['UNIT_ID'].isin(top_8_units)].groupby(['YEAR_', 'UNIT_ID']).agg({
    'OBJECTID': 'count',
    'GIS_ACRES': 'sum'
}).rename(columns={'OBJECTID': 'fire_count', 'GIS_ACRES': 'acres_burned'}).reset_index()

# Use a colormap for units
unit_colors = plt.cm.tab10(np.linspace(0, 1, len(top_8_units)))

fig, axes = plt.subplots(2, 1, figsize=(16, 12))

# Top: Fire Count by Unit Over Time
ax1 = axes[0]
for i, unit in enumerate(top_8_units):
    data = unit_yearly[unit_yearly['UNIT_ID'] == unit]
    label = f"{unit} - {unit_labels.get(unit, unit)}"
    ax1.plot(data['YEAR_'], data['fire_count'], 
             label=label, color=unit_colors[i], linewidth=2, alpha=0.8)

ax1.set_xlabel('Year', fontsize=12)
ax1.set_ylabel('Number of Fires', fontsize=12)
ax1.set_title(f'Fire Count Trends by Unit\nData: {min_year_hq}-{max_year_hq}', fontsize=14, fontweight='bold')
ax1.legend(loc='upper left', fontsize=9, ncol=2, title='Unit ID - Name')
ax1.grid(True, alpha=0.3)
ax1.set_xlim(min_year_hq, max_year_hq)

# Bottom: Burned Acres by Unit Over Time
ax2 = axes[1]
for i, unit in enumerate(top_8_units):
    data = unit_yearly[unit_yearly['UNIT_ID'] == unit]
    label = f"{unit} - {unit_labels.get(unit, unit)}"
    ax2.plot(data['YEAR_'], data['acres_burned'] / 1e3, 
             label=label, color=unit_colors[i], linewidth=2, alpha=0.8)

ax2.set_xlabel('Year', fontsize=12)
ax2.set_ylabel('Thousand Acres Burned', fontsize=12)
ax2.set_title(f'Burned Acres Trends by Unit\nData: {min_year_hq}-{max_year_hq}', fontsize=14, fontweight='bold')
ax2.legend(loc='upper left', fontsize=9, ncol=2, title='Unit ID - Name')
ax2.grid(True, alpha=0.3)
ax2.set_xlim(min_year_hq, max_year_hq)

plt.suptitle(f'CAL FIRE Unit Activity Trends ({min_year_hq}-{max_year_hq})', fontsize=16, fontweight='bold', y=1.01)
plt.tight_layout()

save_path = FIGURES_DIR / 'comprehensive' / '19_unit_trends_over_time.png'
fig.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()
print(f"Figure saved: {save_path}")

---
# Part 9: Executive Summary & ML Readiness

This section consolidates key findings from the analysis and assesses the dataset's readiness for machine learning model development.

**What we've learned:**
- Fire activity has dramatically increased since 2000
- Clear seasonal patterns make temporal features valuable
- Spatial burn frequency shows predictable hotspots
- Agency and unit data provide geographic context
- Data quality is excellent for the 1993+ period

In [ ]:
# Summary Statistics
print("=" * 60)
print("CALIFORNIA FIRE PERIMETER ANALYSIS - EXECUTIVE SUMMARY")
print("=" * 60)

# Dataset overview
print(f"\n📊 DATASET OVERVIEW")
print(f"   Total fire records: {len(gdf):,}")
print(f"   High-quality records (1993+): {len(gdf_hq):,}")
print(f"   Year range: {gdf['YEAR_'].min()} - {gdf['YEAR_'].max()}")
print(f"   CRS: EPSG:3310 (California Albers)")

# Temporal findings
print(f"\n📅 TEMPORAL PATTERNS")
print(f"   Worst fire year: {int(worst_year)} ({worst_acres/1e6:.2f}M acres)")
print(f"   Fire activity accelerated after: 2000")
print(f"   Total acres burned (1993+): {gdf_hq['GIS_ACRES'].sum()/1e6:.1f} million")

# Seasonal findings
print(f"\n🌡️ SEASONAL PATTERNS")
high_risk_stats = gdf_hq[gdf_hq['FIRE_SEASON'] == 'High Risk Season']
print(f"   High Risk Season (Jun-Sep): {len(high_risk_stats)/len(gdf_hq)*100:.1f}% of fires")
print(f"   High Risk burned area: {high_risk_stats['GIS_ACRES'].sum()/gdf_hq['GIS_ACRES'].sum()*100:.1f}% of total")

# Size distribution
print(f"\n📏 FIRE SIZE DISTRIBUTION")
print(f"   Median fire size: {gdf_hq['GIS_ACRES'].median():,.0f} acres")
print(f"   Largest fire: {gdf_hq['GIS_ACRES'].max():,.0f} acres")
print(f"   Mega-fires (>100K): {len(gdf_hq[gdf_hq['GIS_ACRES'] >= 100000]):,}")

# Spatial findings
print(f"\n🗺️ SPATIAL PATTERNS")
print(f"   Cells burned at least once: {(burn_frequency > 0).sum():,}")
print(f"   Max burn frequency: {burn_frequency.max():.0f} times")
print(f"   Grid resolution: {resolution}m")

print("\n" + "=" * 60)
print("ML MODEL READINESS: ✅ READY FOR PHASE 2")
print("=" * 60)

## ML Model Recommendations

### Data Quality Assessment

| Criterion | Status | Evidence from Analysis |
|-----------|--------|------------------------|
| Sufficient history | ✅ | 30+ years (1993-present) |
| Spatial accuracy | ✅ | GPS-based perimeters since 1993 |
| Attribute completeness | ✅ | >90% for CAUSE, AGENCY, DATES, SIZE |
| Temporal coverage | ✅ | All years, all seasons represented |
| Geographic coverage | ✅ | All California regions included |

### Feature Engineering Opportunities

Based on this analysis, the following features show predictive potential:

**From Fire Perimeter Data (This Notebook)**
- Historical burn frequency (cumulative risk map)
- Time since last fire at location
- Fire season (High Risk / Transition / Low Risk)
- Month and day of year
- Agency jurisdiction (CAL FIRE, USDA Forest Service, etc.)
- Unit ID (administrative region)

**To Add in Phase 3**
- Climate: Temperature, precipitation, VPD, drought indices
- Topography: Elevation, slope, aspect (DEM)
- Fuel: Vegetation type, density, fuel moisture
- Human: Roads, population density, infrastructure proximity

### Model Architecture Recommendations

| Component | Recommendation | Rationale |
|-----------|---------------|-----------|
| Grid Resolution | 800m × 800m | Balances detail vs computational cost |
| Target Variable | Binary (burned/not-burned) | Simplifies initial model |
| Temporal Unit | Monthly or seasonal | Matches fire season patterns |
| Architecture | CNN + LSTM | Captures spatial and temporal patterns |
| Loss Function | Focal Loss | Handles extreme class imbalance |
| Validation | Temporal split | Train: 1993-2019, Test: 2020+ |

### Known Challenges

1. **Class Imbalance**: ~99% of grid cells never burn in any given year
2. **Non-Stationarity**: Climate change means past patterns may not perfectly predict future
3. **Rare Events**: Mega-fires are rare but cause most damage
4. **Data Gaps**: Pre-1993 data has quality issues

### Recommended Phase 2 Tasks

1. **Grid Creation**: Generate 800m × 800m grid in EPSG:3310
2. **State Mask**: Exclude water bodies, ocean, out-of-state areas
3. **Rasterization**: Convert fire perimeters to binary grid cells
4. **Temporal Aggregation**: Create monthly/seasonal/annual burn layers
5. **Feature Stack**: Prepare burn frequency as first predictor layer

---
# Conclusion

## Summary of Findings

This comprehensive analysis of **22,000+ fire perimeters spanning 147 years** reveals critical insights about California wildfire patterns:

### Key Findings

| Finding | Evidence |
|---------|----------|
| **Wildfires are accelerating** | Fire count and burned area both show dramatic increase post-2000 |
| **2020 was the worst year** | Record-breaking 4.2 million acres burned in a single year |
| **Clear seasonal pattern** | ~84% of burned area occurs June-September (High Risk Season) |
| **Extreme fire concentration** | Top 1% of fires account for ~58% of total burned area; Top 10% account for ~93% |
| **Geographic hotspots exist** | Northern CA and Sierra foothills show highest burn frequency |
| **Most causes are unknown** | Unknown/Unidentified is the largest category (~30%) due to investigation difficulty |
| **Data quality is excellent** | 1993+ data has >97% completeness for key fields |

### Visualizations Created

This notebook generated **19 figures** including:
- Temporal trend analysis (1900-present)
- Fire Clock polar visualization
- Seasonal heatmaps
- Fire size distributions and Pareto analysis
- Cumulative burn frequency map on basemap
- Agency and Unit activity analysis
- Interactive Folium map of mega-fires

### Implications for ML Model

The analysis confirms that the CAL FIRE perimeter dataset is **well-suited for machine learning**:

1. **Sufficient history**: 30+ years of high-quality data (1993-present)
2. **Clear patterns**: Temporal, seasonal, and spatial patterns are learnable
3. **Feature opportunities**: Historical burn frequency, seasonality, agency/unit
4. **Known challenges**: Class imbalance (most areas don't burn), non-stationarity

---

## Next Steps: Phase 2

The cumulative fire risk map demonstrates the foundational concept for our spatial-temporal prediction model:

1. **Grid Creation**: Generate 800m × 800m grid in EPSG:3310
2. **Rasterization**: Convert fire perimeters to binary burned/not-burned cells
3. **Feature Integration**: Add climate, topography, and vegetation data
4. **Model Development**: Train CNN/LSTM architecture for risk prediction

---

*Analysis prepared for the Tam Air Club Wildfire Prediction Project*  
*In collaboration with UCSF, UCI, and CAL FIRE*  
*Data source: CAL FIRE FRAP Historical Fire Perimeters*  
*Analysis period: 1993-2025 (High-Quality GPS Era)*